# Preprocessamento:

+ Criar variável que identifique o anotador

+ Mover tags que começam com token ' ' (vazio)

+ Remover linhas com '\n' seguidos

+ REGEX:
    + Garantir letra e números onde tamanho for maior que 1.
    + Passar múltiplos símbolos para outra linha. Exemplo:  §3º -->  § \n 3 \n º
    + Remover pontuação de centenas dos números. Exemplo: 12.200 --> 12200

+ Visualização das sentenças com displacy (from spacy import displacy) 

+ Incluir POS tagging.

+ Tranformar o dado $x_i$ em um $x'_i$ que incorpora os 2 últimos e próximos tokens.

# Classificador

+ Visualização: Separar o conjunto de test em 2 ou 3 arquivos e visualizar o que o modelo classificou e o que os anotadores classificaram (separar por id do anotador).

+ Parâmetros utilizados no classificador.

+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

+ Para criar um contexto no erro imprimir 10 palavras antes e depois de dois erros.


# Instruções para os anotadores:

+ Atentar à marcação de tags que envolve espaço 

+ Atentar para não incluir espaço no início da Tag

# Organização do diretório: 

+ Manter toda a análise em somente um diretório

+ Formato do diretório com os datasets: /resources/dataset/

+ Notebooks:
    + '01 - Processamento.ipynb'
        + Gerar 'treino.csv' e teste.csv' para processamento
    + '02 - [CRF].ipynb' - Criando o modelo
        + Usar arquivo dos dados preprocessados gerado pelo notebook 1.
        + Gerar modelo (xxx.model)
    + '03 - Metricas.ipynb'

In [1]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
from collections import Counter

import scipy.stats
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
# Encontra todos os csv's dentro das pastas de 'mock'

extension = 'csv'
all_filenames = [i for i in glob.glob('mock/*/**/***/****.{}'.format(extension))]

In [3]:
all_filenames[:3]

['mock/171300044/[PRATICA_ETAPA_1]/Documentos/20090520_Rcl_5133_601151.ner.csv',
 'mock/171300044/[PRATICA_ETAPA_1]/Documentos/20090520_Rcl_4879_603324.ner.csv',
 'mock/171300044/[PRATICA_ETAPA_1]/Documentos/20130318_RE_542863_128236196.ner.csv']

# Preprocessamento 

In [4]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ'] # Altera a primeira e ultima Tag desse csv
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
combined_csv = pd.concat(frames).reset_index(drop=True)
combined_csv.to_csv("combined_csv.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
combined_csv['Token'] = combined_csv['Token'].astype('str')

In [5]:
combined_csv.head(15), combined_csv.tail(20)

(            Token         Tag
 0          ~~e»to  INICIO_ARQ
 1                           O
 2          c;JJ;~           O
 3               .           O
 4             ~wa           O
 5               /           O
 6                           O
 7            pfi~           O
 8               /           O
 9             <'d           O
 10                          O
 11             \n           O
 12             \n           O
 13  Coordenadoria           O
 14                          O,
                                  Token       Tag
 713274                                         O
 713275                         outubro         O
 713276                                         O
 713277                              de         O
 713278                                         O
 713279                            2018         O
 713280                               .         O
 713281                                         O
 713282                              \n         O
 71

In [6]:
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 713294


In [7]:
combined_csv[-10:] # Conferindo se o index foi resetado

,Token,Tag
713284,Ministro,B_Pessoa
713285,,I_Pessoa
713286,GILMAR,I_Pessoa
713287,,I_Pessoa
713288,MENDES,I_Pessoa
713289,\n,I_Pessoa
713290,id,O
713291,:,O
713292,,O
713293,20181018_HC_162764_15338864359,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [8]:
a_df = combined_csv #Simplifica o nome do arquivo para a função nao ficar grande demais.
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 7729


In [9]:
%%time

combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 

combined_csv.head(), combined_csv.tail()

CPU times: user 3min 14s, sys: 36.8 ms, total: 3min 14s
Wall time: 3min 14s


(    Token         Tag  Sentence #
 0  ~~e»to  INICIO_ARQ  Sentence 1
 1                   O  Sentence 1
 2  c;JJ;~           O  Sentence 1
 3       .           O  Sentence 1
 4     ~wa           O  Sentence 1,
                                  Token       Tag     Sentence #
 713289                              \n  I_Pessoa  Sentence 7730
 713290                              id         O  Sentence 7730
 713291                               :         O  Sentence 7730
 713292                                         O  Sentence 7730
 713293  20181018_HC_162764_15338864359   FIM_ARQ  Sentence 7730)

In [10]:
# Número de sentenças
len(combined_csv['Sentence #'].unique())

7730

# Atualiza a Tag que termina com 'Doutrinador' para 'Doutrina'

In [11]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Precedente', 'I_Precedente', 'B_Pessoa',
       'I_Pessoa', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Doutrina', 'I_Doutrina', 'FIM_ARQ', 'B_Doutrinador',
       'I_Doutrinador'], dtype=object)

In [12]:
# Strip o final 'dor' de todo o DataFrame (formato extremo)
# combined_csv.Tag = combined_csv.Tag.str.rstrip('dor')

In [13]:
# indices de onde a tag ocorre
indx = combined_csv[combined_csv.Tag.str.endswith('Doutrinador')].index.values

### Rodar apenas uma vez

In [14]:
%%time
# Demorado e custoso
for i in range(len(indx)):
    combined_csv.Tag.iloc[indx[i]] = combined_csv.Tag.iloc[indx[i]].rstrip('dor')

CPU times: user 2min 34s, sys: 12 ms, total: 2min 34s
Wall time: 2min 34s


In [15]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Precedente', 'I_Precedente', 'B_Pessoa',
       'I_Pessoa', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Doutrina', 'I_Doutrina', 'FIM_ARQ'], dtype=object)

# Remove enter duplo depois de criar as sentenças

In [16]:
# Teste para ver os casoso onde ocorre enter duplo.
for i in range(len(starts)):
    print(combined_csv.iloc[starts[i]:starts[i]+2][:5])

   Token Tag  Sentence #
11    \n   O  Sentence 1
12    \n   O  Sentence 1
   Token Tag  Sentence #
47    \n   O  Sentence 2
48    \n   O  Sentence 2
   Token Tag  Sentence #
61    \n   O  Sentence 3
62    \n   O  Sentence 3
   Token Tag  Sentence #
67    \n   O  Sentence 4
68    \n   O  Sentence 4
   Token Tag  Sentence #
71    \n   O  Sentence 5
72    \n   O  Sentence 5
   Token           Tag  Sentence #
81    \n  I_Precedente  Sentence 6
82    \n             O  Sentence 6
    Token Tag  Sentence #
174    \n   O  Sentence 7
175    \n   O  Sentence 7
    Token Tag  Sentence #
209    \n   O  Sentence 8
210    \n   O  Sentence 8
    Token Tag  Sentence #
225    \n   O  Sentence 9
226    \n   O  Sentence 9
    Token Tag   Sentence #
240    \n   O  Sentence 10
241    \n   O  Sentence 10
    Token Tag   Sentence #
248    \n   O  Sentence 11
249    \n   O  Sentence 11
    Token           Tag   Sentence #
268    \n  I_Precedente  Sentence 12
269    \n  I_Precedente  Sentence 12
    Token    

7320    \n  I_Pessoa  Sentence 155
     Token Tag    Sentence #
7366    \n   O  Sentence 156
7367    \n   O  Sentence 156
     Token Tag    Sentence #
7402    \n   O  Sentence 157
7403    \n   O  Sentence 157
     Token Tag    Sentence #
7416    \n   O  Sentence 158
7417    \n   O  Sentence 158
     Token Tag    Sentence #
7424    \n   O  Sentence 159
7425    \n   O  Sentence 159
     Token           Tag    Sentence #
7432    \n  I_Precedente  Sentence 160
7433    \n             O  Sentence 160
     Token Tag    Sentence #
7509    \n   O  Sentence 161
7510    \n   O  Sentence 161
     Token Tag    Sentence #
7532    \n   O  Sentence 162
7533    \n   O  Sentence 162
     Token Tag    Sentence #
7548    \n   O  Sentence 163
7549    \n   O  Sentence 163
     Token Tag    Sentence #
7571    \n   O  Sentence 164
7572    \n   O  Sentence 164
     Token Tag    Sentence #
7592    \n   O  Sentence 165
7593    \n   O  Sentence 165
     Token Tag    Sentence #
7611    \n   O  Sentence 166
7612   

      Token                 Tag    Sentence #
17231    \n  I_Ref. Legislativa  Sentence 315
17232    \n  I_Ref. Legislativa  Sentence 315
      Token Tag    Sentence #
17249    \n   O  Sentence 316
17250    \n   O  Sentence 316
      Token Tag    Sentence #
17269    \n   O  Sentence 317
17270    \n   O  Sentence 317
      Token Tag    Sentence #
17283    \n   O  Sentence 318
17284    \n   O  Sentence 318
      Token Tag    Sentence #
17288    \n   O  Sentence 319
17289    \n   O  Sentence 319
      Token Tag    Sentence #
17365    \n   O  Sentence 320
17366    \n   O  Sentence 320
      Token Tag    Sentence #
17435    \n   O  Sentence 321
17436    \n   O  Sentence 321
      Token Tag    Sentence #
17519    \n   O  Sentence 322
17520    \n   O  Sentence 322
      Token Tag    Sentence #
17667    \n   O  Sentence 323
17668    \n   O  Sentence 323
      Token Tag    Sentence #
17751    \n   O  Sentence 324
17752    \n   O  Sentence 324
      Token Tag    Sentence #
17855    \n   O  Sente

      Token Tag    Sentence #
31353    \n   O  Sentence 476
31354    \n   O  Sentence 476
      Token Tag    Sentence #
31854    \n   O  Sentence 477
31855    \n   O  Sentence 477
      Token Tag    Sentence #
31945    \n   O  Sentence 478
31946    \n   O  Sentence 478
      Token Tag    Sentence #
31992    \n   O  Sentence 479
31993    \n   O  Sentence 479
      Token Tag    Sentence #
32585    \n   O  Sentence 480
32586    \n   O  Sentence 480
      Token Tag    Sentence #
32660    \n   O  Sentence 481
32661    \n   O  Sentence 481
      Token Tag    Sentence #
32684    \n   O  Sentence 482
32685    \n   O  Sentence 482
      Token Tag    Sentence #
33027    \n   O  Sentence 483
33028    \n   O  Sentence 483
      Token Tag    Sentence #
33071    \n   O  Sentence 484
33072    \n   O  Sentence 484
      Token                 Tag    Sentence #
33534    \n  I_Ref. Legislativa  Sentence 485
33535    \n  I_Ref. Legislativa  Sentence 485
      Token Tag    Sentence #
33561    \n   O  Sente

      Token Tag    Sentence #
43257    \n   O  Sentence 636
43258    \n   O  Sentence 636
      Token Tag    Sentence #
43371    \n   O  Sentence 637
43372    \n   O  Sentence 637
      Token Tag    Sentence #
43521    \n   O  Sentence 638
43522    \n   O  Sentence 638
      Token Tag    Sentence #
43534    \n   O  Sentence 639
43535    \n   O  Sentence 639
      Token Tag    Sentence #
43560    \n   O  Sentence 640
43561    \n   O  Sentence 640
      Token Tag    Sentence #
43644    \n   O  Sentence 641
43645    \n   O  Sentence 641
      Token Tag    Sentence #
43723    \n   O  Sentence 642
43724    \n   O  Sentence 642
      Token Tag    Sentence #
43843    \n   O  Sentence 643
43844    \n   O  Sentence 643
      Token Tag    Sentence #
43921    \n   O  Sentence 644
43922    \n   O  Sentence 644
      Token Tag    Sentence #
43984    \n   O  Sentence 645
43985    \n   O  Sentence 645
      Token Tag    Sentence #
44065    \n   O  Sentence 646
44066    \n   O  Sentence 646
      Toke

...

       Token Tag     Sentence #
639470    \n   O  Sentence 7012
639471    \n   O  Sentence 7012
       Token Tag     Sentence #
639575    \n   O  Sentence 7013
639576    \n   O  Sentence 7013
       Token Tag     Sentence #
639771    \n   O  Sentence 7014
639772    \n   O  Sentence 7014
       Token Tag     Sentence #
639968    \n   O  Sentence 7015
639969    \n   O  Sentence 7015
       Token Tag     Sentence #
639986    \n   O  Sentence 7016
639987    \n   O  Sentence 7016
       Token Tag     Sentence #
639996    \n   O  Sentence 7017
639997    \n   O  Sentence 7017
       Token Tag     Sentence #
640127    \n   O  Sentence 7018
640128    \n   O  Sentence 7018
       Token Tag     Sentence #
640281    \n   O  Sentence 7019
640282    \n   O  Sentence 7019
       Token Tag     Sentence #
640461    \n   O  Sentence 7020
640462    \n   O  Sentence 7020
       Token Tag     Sentence #
640714    \n   O  Sentence 7021
640715    \n   O  Sentence 7021
       Token Tag     Sentence #
640858  

       Token Tag     Sentence #
660639    \n   O  Sentence 7174
660640    \n   O  Sentence 7174
       Token Tag     Sentence #
661021    \n   O  Sentence 7175
661022    \n   O  Sentence 7175
       Token Tag     Sentence #
661186    \n   O  Sentence 7176
661187    \n   O  Sentence 7176
       Token Tag     Sentence #
661277    \n   O  Sentence 7177
661278    \n   O  Sentence 7177
       Token Tag     Sentence #
661422    \n   O  Sentence 7178
661423    \n   O  Sentence 7178
       Token Tag     Sentence #
661669    \n   O  Sentence 7179
661670    \n   O  Sentence 7179
       Token Tag     Sentence #
661750    \n   O  Sentence 7180
661751    \n   O  Sentence 7180
       Token Tag     Sentence #
661966    \n   O  Sentence 7181
661967    \n   O  Sentence 7181
       Token Tag     Sentence #
662032    \n   O  Sentence 7182
662033    \n   O  Sentence 7182
       Token Tag     Sentence #
662274    \n   O  Sentence 7183
662275    \n   O  Sentence 7183
       Token Tag     Sentence #
662354  

       Token Tag     Sentence #
677849    \n   O  Sentence 7334
677850    \n   O  Sentence 7334
       Token Tag     Sentence #
677855    \n   O  Sentence 7335
677856    \n   O  Sentence 7335
       Token           Tag     Sentence #
677876    \n  I_Precedente  Sentence 7336
677877    \n             O  Sentence 7336
       Token Tag     Sentence #
677886    \n   O  Sentence 7337
677887    \n   O  Sentence 7337
       Token Tag     Sentence #
677979    \n   O  Sentence 7338
677980    \n   O  Sentence 7338
       Token Tag     Sentence #
678087    \n   O  Sentence 7339
678088    \n   O  Sentence 7339
       Token Tag     Sentence #
678318    \n   O  Sentence 7340
678319    \n   O  Sentence 7340
       Token Tag     Sentence #
678523    \n   O  Sentence 7341
678524    \n   O  Sentence 7341
       Token Tag     Sentence #
678941    \n   O  Sentence 7342
678942    \n   O  Sentence 7342
       Token Tag     Sentence #
679010    \n   O  Sentence 7343
679011    \n   O  Sentence 7343
       Tok

693917    \n   O  Sentence 7494
       Token Tag     Sentence #
693959    \n   O  Sentence 7495
693960    \n   O  Sentence 7495
       Token Tag     Sentence #
694030    \n   O  Sentence 7496
694031    \n   O  Sentence 7496
       Token Tag     Sentence #
694056    \n   O  Sentence 7497
694057    \n   O  Sentence 7497
       Token Tag     Sentence #
694066    \n   O  Sentence 7498
694067    \n   O  Sentence 7498
       Token Tag     Sentence #
694170    \n   O  Sentence 7499
694171    \n   O  Sentence 7499
       Token Tag     Sentence #
694176    \n   O  Sentence 7500
694177    \n   O  Sentence 7500
       Token Tag     Sentence #
694190    \n   O  Sentence 7501
694191    \n   O  Sentence 7501
       Token Tag     Sentence #
694204    \n   O  Sentence 7502
694205    \n   O  Sentence 7502
       Token Tag     Sentence #
694315    \n   O  Sentence 7503
694316    \n   O  Sentence 7503
       Token Tag     Sentence #
694448    \n   O  Sentence 7504
694449    \n   O  Sentence 7504
       T

       Token Tag     Sentence #
706065    \n   O  Sentence 7653
706066    \n   O  Sentence 7653
       Token Tag     Sentence #
706210    \n   O  Sentence 7654
706211    \n   O  Sentence 7654
       Token Tag     Sentence #
706312    \n   O  Sentence 7655
706313    \n   O  Sentence 7655
       Token Tag     Sentence #
706375    \n   O  Sentence 7656
706376    \n   O  Sentence 7656
       Token Tag     Sentence #
706659    \n   O  Sentence 7657
706660    \n   O  Sentence 7657
       Token Tag     Sentence #
706821    \n   O  Sentence 7658
706822    \n   O  Sentence 7658
       Token Tag     Sentence #
706970    \n   O  Sentence 7659
706971    \n   O  Sentence 7659
       Token Tag     Sentence #
707121    \n   O  Sentence 7660
707122    \n   O  Sentence 7660
       Token Tag     Sentence #
707356    \n   O  Sentence 7661
707357    \n   O  Sentence 7661
       Token Tag     Sentence #
707421    \n   O  Sentence 7662
707422    \n   O  Sentence 7662
       Token Tag     Sentence #
707481  

In [17]:
# Cria um array com as posições a serem retiradas.
pos = []
for i in range(len(starts)):
    pos.append(starts[i])
    pos.append(starts[i]+1)
pos[:5]

[11, 12, 47, 48, 61]

In [18]:
# Remove as linhas do dataframe e reseta os índices.
combined_csv = combined_csv.drop(pos).reset_index(drop=True)

In [19]:
# Confirma se a remoção foi bem sucedida.
combined_csv.head(15)

,Token,Tag,Sentence #
0,~~e»to,INICIO_ARQ,Sentence 1
1,,O,Sentence 1
2,c;JJ;~,O,Sentence 1
3,.,O,Sentence 1
4,~wa,O,Sentence 1
5,/,O,Sentence 1
6,,O,Sentence 1
7,pfi~,O,Sentence 1
8,/,O,Sentence 1
9,<'d,O,Sentence 1


# Mover B_  com Token vazio para linha abaixo

In [20]:
# 'begins' identifica as situações onde a Tag começa com 'B_' e o Token é vazio, uma situação onde 
# o anotador começou a marcação de um espaço vazio gerando a inconsistância.

begins = combined_csv[(combined_csv['Token']==' ') & 
                      (combined_csv['Tag'].str.startswith('B_'))].index.values

In [21]:
begins[0], len (begins)

(3654, 153)

In [22]:
%%time
# Rodar apenas uma vez
for i in range(len(begins)):
    combined_csv.Tag.iloc[begins[i]+1] = combined_csv.Tag.iloc[begins[i]] #Acertar a Tag do Token para começar
                                                                          #sem espaço
    combined_csv.Tag.iloc[begins[i]] = 'O' #Marca o espaço vazio como 'O'

CPU times: user 5.21 s, sys: 0 ns, total: 5.21 s
Wall time: 5.21 s


In [23]:
i, n = 1, 10
combined_csv.iloc[begins[i]-n+7:begins[i]+n]

,Token,Tag,Sentence #
5199,redação,O,Sentence 114
5200,,O,Sentence 114
5201,da,O,Sentence 114
5202,,O,Sentence 114
5203,L,B_Ref. Legislativa,Sentence 114
5204,.,I_Ref. Legislativa,Sentence 114
5205,,I_Ref. Legislativa,Sentence 114
5206,10358/01,I_Ref. Legislativa,Sentence 114
5207,,I_Ref. Legislativa,Sentence 114
5208,abrange,O,Sentence 114


# Tratar marcações que incluem vírgula no final da marcação

In [41]:
df_teste = combined_csv.copy()

In [52]:
# df_teste.iloc[2].shift(+1)

df_teste['Token'].shift(-1)

0                                       
1                                 c;JJ;~
2                                      .
3                                    ~wa
4                                      /
                       ...              
697831                                id
697832                                 :
697833                                  
697834    20181018_HC_162764_15338864359
697835                               NaN
Name: Token, Length: 697836, dtype: object

In [51]:
df_teste.iloc[2]

Token             c;JJ;~
Tag                    O
Sentence #    Sentence 1
Name: 2, dtype: object

In [114]:
# Índice das posições onde ocorre o fim da marcação em uma vírgula
inx = df_teste[(df_teste.Tag.str.startswith('I')) & (df_teste.Token == ',') & (df_teste.Tag.shift(-1) == 'O')].index.values

In [145]:
# Conferindo a ocorrência
i = 9
df_teste.iloc[inx[i]-5:inx[i]+10]

,Token,Tag,Sentence #
2497,,I_Precedente,Sentence 71
2498,n,I_Precedente,Sentence 71
2499,.,I_Precedente,Sentence 71
2500,,I_Precedente,Sentence 71
2501,2006.38.00.744462-0,I_Precedente,Sentence 71
2502,",",I_Precedente,Sentence 71
2503,,O,Sentence 71
2504,com,O,Sentence 71
2505,,O,Sentence 71
2506,fundamento,O,Sentence 71


In [146]:
df_teste = df_teste.drop(inx).reset_index(drop=True)

In [148]:
# Conferindo se funcionou
i = 9
df_teste.iloc[inx[i]-20:inx[i]+15]

,Token,Tag,Sentence #
2482,,O,Sentence 71
2483,autos,O,Sentence 71
2484,,O,Sentence 71
2485,do,O,Sentence 71
2486,,O,Sentence 71
2487,Processo,B_Precedente,Sentence 71
2488,,I_Precedente,Sentence 71
2489,n,I_Precedente,Sentence 71
2490,.,I_Precedente,Sentence 71
2491,,I_Precedente,Sentence 71


In [149]:
combined_csv = df_teste.copy()

In [152]:
combined_csv[(combined_csv.Tag.str.startswith('B')) & (combined_csv.Token == ' ')]

,Token,Tag,Sentence #


In [154]:
list(combined_csv.Tag.unique())

['INICIO_ARQ',
 'O',
 'B_Precedente',
 'I_Precedente',
 'B_Pessoa',
 'I_Pessoa',
 'B_Ref. Legislativa',
 'I_Ref. Legislativa',
 'B_Doutrina',
 'I_Doutrina',
 'FIM_ARQ']

# IMPORTANTE: Esse último passo é necessário para salvar todas as alterações feitas no preprocessamento.

In [155]:
# Salvando o processamento feito nos dados
combined_csv.to_csv("preprocessados.csv",index=False,encoding='utf-8')

# DUMP

# Tratar caracteres especiais

In [156]:
df_teste = combined_csv.copy()

In [157]:
# Identifica os tokens que começam com '§' e são maiores que 1 pois queremos pegar os casos onde
# '§' está associado ao numero do paragrafo.
espec_carac = list(df_teste[(df_teste.Token.str.startswith('§')) & (df_teste.Token.str.len()>1)].Token.unique())
espec_carac

['§2º', '§4º', '§§', '§3º', '§1º', '§1°', '§7º']

In [158]:
df_teste[df_teste.Token.str.match(espec_carac[0])]

,Token,Tag,Sentence #
15765,§2º,I_Ref. Legislativa,Sentence 310
282334,§2º,B_Ref. Legislativa,Sentence 3412
288581,§2º,I_Ref. Legislativa,Sentence 3464
345473,§2º,I_Ref. Legislativa,Sentence 3924
444852,§2º,I_Ref. Legislativa,Sentence 5267
574861,§2º,I_Ref. Legislativa,Sentence 6563
615705,§2º,B_Ref. Legislativa,Sentence 6948
621949,§2º,I_Ref. Legislativa,Sentence 7000


## Teste com duas sentenças

In [159]:
df_senten = df_teste[df_teste['Sentence #'] == 'Sentence 310'].append(df_teste[df_teste['Sentence #'] == 'Sentence 1589'])

df_senten.reset_index(drop=True,inplace=True)

In [160]:
for i in range(len(df_senten)):
    if any(carac in df_senten.Token.iloc[i] for carac in espec_carac) == True:
        line = df_senten.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

In [161]:
(df_senten1.tail(10))

,Token,Tag,Sentence #
289,§,I_Ref. Legislativa,Sentence 1589
290,4,I_Ref. Legislativa,Sentence 1589
291,º,I_Ref. Legislativa,Sentence 1589
292,",",I_Ref. Legislativa,Sentence 1589
293,,I_Ref. Legislativa,Sentence 1589
294,do,I_Ref. Legislativa,Sentence 1589
295,,I_Ref. Legislativa,Sentence 1589
296,CPC,I_Ref. Legislativa,Sentence 1589
297,.,I_Ref. Legislativa,Sentence 1589
298,,O,Sentence 1589


In [162]:
df_senten.tail(10)

,Token,Tag,Sentence #
287,",",I_Ref. Legislativa,Sentence 1589
288,,I_Ref. Legislativa,Sentence 1589
289,§4º,I_Ref. Legislativa,Sentence 1589
290,",",I_Ref. Legislativa,Sentence 1589
291,,I_Ref. Legislativa,Sentence 1589
292,do,I_Ref. Legislativa,Sentence 1589
293,,I_Ref. Legislativa,Sentence 1589
294,CPC,I_Ref. Legislativa,Sentence 1589
295,.,I_Ref. Legislativa,Sentence 1589
296,,O,Sentence 1589


## Para o conjunto de teste (com todas as sentenças)

In [39]:
for i in range(len(df_teste)):
    if any(carac in df_teste.Token.iloc[i] for carac in espec_carac) == True:
        line = df_teste.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

In [40]:
df_senten1[(df_senten1.Token.str.startswith('§')) & (df_senten1.Token.str.len()>1)]

,Token,Tag,Sentence #
15818,§2º,I_Ref. Legislativa,Sentence 310
131173,§4º,I_Ref. Legislativa,Sentence 1589
149864,§§,I_Ref. Legislativa,Sentence 1789
150253,§§,B_Ref. Legislativa,Sentence 1790
178152,§4º,I_Ref. Legislativa,Sentence 2196
178238,§3º,I_Ref. Legislativa,Sentence 2197
179601,§4º,I_Ref. Legislativa,Sentence 2216
183944,§1º,B_Ref. Legislativa,Sentence 2272
184458,§3º,I_Ref. Legislativa,Sentence 2276
186230,§3º,I_Ref. Legislativa,Sentence 2287


In [32]:
df_senten1

,Token,Tag,Sentence #
0,~~e»to,INICIO_ARQ,Sentence 1
1,,O,Sentence 1
2,c;JJ;~,O,Sentence 1
3,.,O,Sentence 1
4,~wa,O,Sentence 1
...,...,...,...
697833,\n,I_Pessoa,Sentence 7730
697834,id,O,Sentence 7730
697835,:,O,Sentence 7730
697836,,O,Sentence 7730


# Olhando para a predição de cada X

In [ ]:
X_test[0][0]['word.lower()'], y_pred[0][0], y_test[0][0]

In [ ]:
# Cria dataframe com o que o modelo predisse para análise.

result = []
for i in range(len(y_test)):
    df = pd.DataFrame(zip([X_test[i][j]['word.lower()'] for j in range(len(X_test[i]))], y_test[i], y_pred[i]))
#     df = pd.DataFrame(zip(X_test[i], y_test[i], y_pred[i]))
    result.append(df)
    
result = pd.concat(result)
result.reset_index(inplace=True, drop=True)

In [ ]:
# combined_csv = pd.concat(frames)
result.to_csv("resultado.csv", index=False, encoding='utf-8')

In [ ]:
result.shape

In [ ]:
result.columns = ['X_test','y_test', 'y_pred']
result.head()

In [ ]:
result[result['y_test'] != result['y_pred']]

In [ ]:
print('Proporção de erros:',result[result.y_test != result.y_pred].shape[0] / result.shape[0])

----------------------------------------------------------------------------------------

In [ ]:
df_res = pd.read_csv('mock/resultado.csv')
df_combined = pd.read_csv('mock/combined_csv.csv')

In [ ]:
df_res.head()

In [ ]:
df_res.shape

In [ ]:
df_erros = df_res[df_res['y_test'] != df_res['y_pred']]
df_erros.head()

In [ ]:
df_erros[(df_erros['X_test']==' ') & (df_erros['y_test'].str.startswith('B_'))]
indices = df_erros[(df_erros['X_test']==' ') & (df_erros['y_test'].str.startswith('B_'))].index.values

In [ ]:
n = 10
pos = indices[1]

print('Retorna %d linhas antes e depois do ocorrido na linha %d para tag que começa com B_'%(n,pos)),df_res.iloc[pos-n:pos+n]

In [ ]:
df_res

In [ ]:
for i in indices:
    df.iloc[pos-n:pos+n]

# Remove pontuação dos números

In [33]:
5=4

SyntaxError: can't assign to literal (<ipython-input-33-d9863e100aa5>, line 1)

In [ ]:
# Criando um DataFrame onde todas os tokens possuem pontuação 
df_pont = combined_csv[(combined_csv.Token.str.contains("""[.]""")) & (combined_csv.Token.str.len()>1)]

In [ ]:
df_pont.shape

In [ ]:
# Tokens de tamanho maior que 1.
df_pont.head(30)

In [ ]:
# combined_csv.iloc[2722-3:2722+5]
df_pont[(df_pont.Tag.str.endswith('Precedente'))]

In [ ]:
df_pont.Tag.value_counts()

In [ ]:
# combined_csv.applymap(np.isreal)
# ord(combined_csv.Token)#<128
# combined_csv.Token[:10]
ord(combined_csv.Token.iloc[0])

In [ ]:
import string
alphabet = string.ascii_letters#+string.punctuation
alphabet 

In [ ]:
regexPattern = re.compile(alphabet)

In [ ]:
combined_csv.Token = combined_csv.Token.astype('str')

In [ ]:
combined_csv.iloc[[3,5]]

In [ ]:
regexPattern.findall(combined_csv.Token.iloc[i])

In [ ]:
# combined_csv.Token[combined_csv.Token.str.contains(alphabet) == False]

# regexPattern.findall(combined_csv.Token)

l = []
for i in range(len(combined_csv.Token)):
#     l.append(regexPattern.findall(combined_csv.Token.iloc[i]).index)
    indx = regexPattern.findall(combined_csv.Token.iloc[i]).index.values
    l.append(indx)
l

In [ ]:
alphabet = string.ascii_letters
mainStr = combined_csv.Token.iloc[0]

# Create a regex pattern to match character 's'
regexPattern = re.compile(alphabet)
 
# Iterate over all the matches of regex pattern
iteratorOfMatchObs = regexPattern.finditer(mainStr)
indexPositions = []
count = 0
for matchObj in iteratorOfMatchObs:
    indexPositions.append(matchObj.start())
    count = count + 1
 
# print("Occurrence Count of character 's' : ", count)
print("Index Positions of 's' are : ", indexPositions)


In [ ]:
l = []
for i in range(len(combined_csv)):
    iteratorOfMatchObs = regexPattern.finditer(combined_csv.Token.iloc[i])
    indexPositions = []
    count = 0
    for matchObj in iteratorOfMatchObs:
        indexPositions.append(matchObj.start())
        count = count + 1
    l.append(indexPositions)

In [ ]:
l.count([])

In [ ]:
regexPattern.findall(combined_csv.Token.iloc[0]).index

In [ ]:
# combined_csv[combined_csv.Token.str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~123456789]""")]

# combined_csv[(combined_csv.Token.str.contains("""[./]"""))& (combined_csv.Token.str.len() > 1)]

In [ ]:
# No meio do preprocessamento foi identificado tokens que terminam com 'adv' e estão colados ao nome do
# advogado referente ao caso.
# Tokens que terminan com 'adv'.

adv = combined_csv[combined_csv.Token.str.endswith('ADV')]#.reset_index(drop=True)
adv.head()#, adv.shape

In [ ]:
# Desconfiança que o nome do advogado está agregado à palavra 'adv'

adv[adv.Token.str.len() >3]
adv_pos = adv[adv.Token.str.len() >3].index.values

k = 2 # Vê as linhas antes e depois do k-ésimo ocorrido .
combined_csv.iloc[adv_pos[k]-3:adv_pos[k]+3] # Olhando para as linhas anteriores e posteriores o acontecimento.
# combined_csv.iloc[adv_pos[0]].Token[-3:] #Separando a parte 'adv'

In [ ]:
print("Quantidade de vezes que esse caso ocorre em todos os arquivos:",len(adv_pos))